In [ ]:
import cv2
import glob
import numpy as np
from skimage.feature import hog, local_binary_pattern
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
import seaborn as sns

# Establecer estilo para gráficos
sns.set(style="whitegrid")

# Definir las carpetas de frutas frescas y podridas
freshfruits = {
    "freshapples": "freshapple",
    "freshbanana": "freshbanana",
    "freshoranges": "freshorange"
}

rottenfruits = {
    "rottenapples": "rottenapple",
    "rottenbanana": "rottenbanana",
    "rottenoranges": "rottenorange"
}

# Parámetros para LBP
LBP_RADIUS = 1
LBP_N_POINTS = 8 * LBP_RADIUS
LBP_METHOD = 'uniform'


In [ ]:
def process_images(folder_dict, base_path, data_list, labels_list):
    for folder, label in folder_dict.items():
        folder_path = f"{base_path}/{folder}/*.png"
        for image_path in glob.glob(folder_path):
            img = cv2.imread(image_path)
            if img is None or len(img.shape) != 3 or img.shape[2] != 3:
                print(f"Imagen no válida o con canales incorrectos: {image_path}")
                continue
            try:
                # Redimensionar la imagen
                resized_img = cv2.resize(img, (64, 32))
                
                # Conversión a escala de grises para HOG y LBP
                gray_img = cv2.cvtColor(resized_img, cv2.COLOR_BGR2GRAY)
                
                # Extracción de características HOG
                hog_features = hog(
                    gray_img,
                    orientations=9,
                    pixels_per_cell=(8, 8),
                    cells_per_block=(2, 2),
                    visualize=False,
                    feature_vector=True,
                    block_norm='L2-Hys'
                )
                
                # Extracción de histogramas de color (B, G, R)
                hist_features = []
                for i in range(3):
                    hist = cv2.calcHist([resized_img], [i], None, [256], [0, 256])
                    hist = cv2.normalize(hist, hist).flatten()
                    hist_features.extend(hist)
                
                # Extracción de características LBP
                lbp = local_binary_pattern(gray_img, LBP_N_POINTS, LBP_RADIUS, LBP_METHOD)
                lbp_hist, _ = np.histogram(lbp.ravel(),
                                           bins=np.arange(0, LBP_N_POINTS + 3),
                                           range=(0, LBP_N_POINTS + 2))
                lbp_hist = lbp_hist.astype("float")
                lbp_hist /= (lbp_hist.sum() + 1e-6)  # Normalizar el histograma
                lbp_features = lbp_hist
                
                # Concatenar características
                features = np.concatenate([hog_features, hist_features, lbp_features])
                
            except Exception as e:
                print(f"Error procesando {image_path}: {e}")
                continue

            # Agregar las características y la etiqueta a las listas
            data_list.append(features)
            labels_list.append(label)


In [ ]:
data_train = []
labels_train = []
data_test = []
labels_test = []

train_base_path = "../Imatges/dataset/train"
test_base_path = "../Imarges/dataset/test"

# Procesar imágenes de entrenamiento
process_images(freshfruits, train_base_path, data_train, labels_train)
process_images(rottenfruits, train_base_path, data_train, labels_train)
print("Total de imágenes de entrenamiento procesadas:", len(data_train))

# Procesar imágenes de prueba
process_images(freshfruits, test_base_path, data_test, labels_test)
process_images(rottenfruits, test_base_path, data_test, labels_test)
print("Total de imágenes de prueba procesadas:", len(data_test))

# Convertir a arrays de NumPy
X_train = np.array(data_train)
y_train_labels = np.array(labels_train)
X_test = np.array(data_test)
y_test_labels = np.array(labels_test)

# Codificar las etiquetas
encoder = LabelEncoder()
encoder.fit(np.concatenate((y_train_labels, y_test_labels)))

y_train = encoder.transform(y_train_labels)
y_test = encoder.transform(y_test_labels)


In [ ]:
# Definir y entrenar el modelo KNN
knn_classifier = KNeighborsClassifier(n_neighbors=5)  # Puedes ajustar n_neighbors
knn_classifier.fit(X_train, y_train)

# Predicción sobre el conjunto de prueba
y_pred_knn = knn_classifier.predict(X_test)

# Evaluación del modelo
accuracy_knn = accuracy_score(y_test, y_pred_knn)
print("----- KNN Classifier -----")
print(f"Accuracy: {accuracy_knn:.4f}\n")
print("Informe de clasificación:")
print(classification_report(y_test, y_pred_knn, target_names=encoder.classes_))


In [ ]:
cm = confusion_matrix(y_test, y_pred_knn)
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt="d", cmap='Blues', 
            xticklabels=encoder.classes_, 
            yticklabels=encoder.classes_)
plt.title("Matriz de Confusión - KNN")
plt.ylabel('Etiqueta verdadera')
plt.xlabel('Etiqueta predicha')
plt.show()
